##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ingeniería de características mediante canalizaciones de TFX y TensorFlow Transform

***Transforme los datos de entrada y entrene un modelo con una canalización de TFX.***

Nota: Recomendamos ejecutar este tutorial en un bloc de notas de Colab, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tft"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_transform.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_transform.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/penguin_transform.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

En este tutorial basado en un bloc de notas, crearemos y ejecutaremos una canalización de TFX para ingerir datos de entrada sin procesar y preprocesarlos adecuadamente para el entrenamiento de aprendizaje automático (ML). Este bloc de notas se basa en la canalización de TFX que creamos en [la validación de datos mediante canalización de TFX y el tutorial de TensorFlow Data Validation](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv). Si aún no ha leído este tutorial, debe leerlo antes de continuar con este bloc de notas.

Puede aumentar la calidad predictiva de sus datos o reducir la dimensionalidad mediante la ingeniería de características. Uno de los beneficios de usar TFX es que escribirá su código de transformación una vez y las transformaciones resultantes serán coherentes entre el entrenamiento y el servicio para evitar sesgos entre entrenamiento y servicio.

Agregaremos un componente `Transform` a la canalización. El componente Transform se implementa utilizando la biblioteca [tf.transform](https://www.tensorflow.org/tfx/transform/get_started).

Consulte [Explicación de las canalizaciones de TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para obtener más información sobre varios conceptos en TFX.

## Preparación

Primero tenemos que instalar el paquete de Python para TFX y descargar el conjunto de datos que usaremos para nuestro modelo.

### Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instalación de TFX


In [ ]:
!pip install -U tfx

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

### ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe hacer clic en el botón "REINICIAR TIEMPO DE EJECUCIÓN" o usar el menú "Tiempo de ejecución &gt; Reiniciar tiempo de ejecución ..." para reiniciar el tiempo de ejecución. Esto se debe a la forma en que Colab carga los paquetes.

Verifique las versiones de TensorFlow y TFX.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### Configuración de variables

Hay algunas variables que se utilizan para definir una canalización. Puede personalizar estas variables como desee. De forma predeterminada, todas las salidas de la canalización se generarán en el directorio actual.

In [ ]:
import os

PIPELINE_NAME = "penguin-transform"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Preparación de datos de ejemplo

Descargaremos el conjunto de datos de ejemplo para usarlo en nuestra canalización de TFX. El conjunto de datos que usamos es el [conjunto de datos Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

Sin embargo, a diferencia de tutoriales anteriores que utilizaban un conjunto de datos ya preprocesado, utilizaremos el conjunto de datos Palmer Penguins **sin procesar**.


Debido a que TFX ExampleGen lee entradas de un directorio, tenemos que crear un directorio y copiar el conjunto de datos en él.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_path = 'https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_path, _data_filepath)

Veamos rápidamente cómo se ven los datos sin procesar.

In [ ]:
!head {_data_filepath}

Hay algunas entradas con valores faltantes que se representan como `NA`. Simplemente eliminaremos esas entradas en este tutorial.

In [ ]:
!sed -i '/\bNA\b/d' {_data_filepath}
!head {_data_filepath}

Debería poder ver siete características que describen a los pingüinos. Usaremos el mismo conjunto de características que en los tutoriales anteriores: 'culmen_length_mm', 'culmen_profundidad_mm', 'flipper_length_mm', 'body_mass_g'; y predeciremos 'species' de un pingüino.

**La única diferencia será que los datos de entrada no se preprocesan.** Tenga en cuenta que no usaremos otras características como "isla" o "sexo" en este tutorial.

### Preparación de un archivo de esquema

Como se describe en [Validación de datos mediante canalización de TFX y Tutorial de TensorFlow Data validation](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv), necesitamos un archivo de esquema para el conjunto de datos. Debido a que el conjunto de datos es diferente al del tutorial anterior, debemos generarlo nuevamente. En este tutorial, omitiremos esos pasos y solo usaremos un archivo de esquema preparado.


In [ ]:
import shutil

SCHEMA_PATH = 'schema'

_schema_uri = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/schema/raw/schema.pbtxt'
_schema_filename = 'schema.pbtxt'
_schema_filepath = os.path.join(SCHEMA_PATH, _schema_filename)

os.makedirs(SCHEMA_PATH, exist_ok=True)
urllib.request.urlretrieve(_schema_uri, _schema_filepath)

Este archivo de esquema se creó con la misma canalización que en el tutorial anterior sin ningún cambio manual.

## Cómo crear una canalización

Las canalizaciones de TFX se definen mediante las API de Python. Agregaremos el componente `Transform` a la canalización que creamos en el [tutorial de validación de datos](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv).

Un componente Transform requiere datos de entrada de un componente `ExampleGen` y un esquema de un componente `SchemaGen`, y produce un "grafo de transformación". La salida se usará en un componente `Trainer`. Opcionalmente, Transform puede producir además "datos transformados", que son los datos materializados después de la transformación. Sin embargo, transformaremos los datos durante el entrenamiento en este tutorial sin la materialización de los datos transformados intermedios.

Una cosa a tener en cuenta es que necesitamos definir una función de Python, `preprocessing_fn`, para describir cómo se deben transformar los datos de entrada. Esto es similar a un componente Trainer que también requiere código de usuario para la definición del modelo.


### Cómo escribir código de preprocesamiento y entrenamiento

Tenemos que definir dos funciones de Python. Una para Transform y otra para Trainer.

#### preprocessing_fn

El componente Transform encontrará una función llamada `preprocessing_fn` en el archivo del módulo dado como lo hicimos con el componente `Trainer`. También puede especificar una función específica utilizando el <a href="https://github.com/tensorflow/tfx/blob/142de6e887f26f4101ded7925f60d7d4fe9d42ed/tfx/components/transform/component.py#L113" data-md-type="link">parámetro `preprocessing_fn`</a> del componente Transform.

En este ejemplo, haremos dos tipos de transformación. Para características numéricas continuas como `culmen_length_mm` y `body_mass_g` , normalizaremos estos valores con ayuda de la función [tft.scale_to_z_score](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score). Para la característica de etiqueta, tenemos que convertir etiquetas de cadena en valores de índice numérico. Usaremos [`tf.lookup.StaticHashTable`](https://www.tensorflow.org/api_docs/python/tf/lookup/StaticHashTable) para la conversión.

Para identificar fácilmente los campos transformados, agregamos un sufijo `_xf` a los nombres de las características transformadas.

#### run_fn

El modelo en sí es casi el mismo que en los tutoriales anteriores, pero esta vez usaremos el grafo de transformación del componente Transform para transformar los datos de entrada.

Otra diferencia importante en comparación con el tutorial anterior es que ahora exportamos un modelo para servir que incluye no solo el grafo de cálculo del modelo, sino también el grafo de transformación para preprocesamiento, que se genera en el componente Transform. Debemos definir una función separada que se utilizará para servir las solicitudes entrantes. Como puede ver, se usó la misma función `_apply_preprocessing` tanto para los datos de entrenamiento como para la solicitud de servicio.


In [ ]:
_module_file = 'penguin_utils.py'

In [ ]:
%%writefile {_module_file}


from typing import List, Text
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

# Specify features that we will use.
_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10


# NEW: TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature.
  """
  outputs = {}

  # Uses features defined in _FEATURE_KEYS only.
  for key in _FEATURE_KEYS:
    # tft.scale_to_z_score computes the mean and variance of the given feature
    # and scales the output based on the result.
    outputs[key] = tft.scale_to_z_score(inputs[key])

  # For the label column we provide the mapping from string to index.
  # We could instead use `tft.compute_and_apply_vocabulary()` in order to
  # compute the vocabulary dynamically and perform a lookup.
  # Since in this example there are only 3 possible values, we use a hard-coded
  # table for simplicity.
  table_keys = ['Adelie', 'Chinstrap', 'Gentoo']
  initializer = tf.lookup.KeyValueTensorInitializer(
      keys=table_keys,
      values=tf.cast(tf.range(len(table_keys)), tf.int64),
      key_dtype=tf.string,
      value_dtype=tf.int64)
  table = tf.lookup.StaticHashTable(initializer, default_value=-1)
  outputs[_LABEL_KEY] = table.lookup(inputs[_LABEL_KEY])

  return outputs


# NEW: This function will apply the same transform operation to training data
#      and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  transformed_features = tft_layer(raw_features)
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  else:
    return transformed_features, None


# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format.
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema includes unnecessary fields like 'species' and
    # 'island', we filter feature_spec to include required keys only.
    required_feature_spec = {
        k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS
    }
    parsed_features = tf.io.parse_example(serialized_tf_examples,
                                          required_feature_spec)

    # Preprocess parsed input with transform operation defined in
    # preprocessing_fn().
    transformed_features, _ = _apply_preprocessing(parsed_features,
                                                   model.tft_layer)
    # Run inference with ML model.
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
      schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=key)
      for key in _FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # NEW: Save a computation graph including transform layer.
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Ahora ha completado todos los pasos de preparación para compilar una canalización de TFX.

### Cómo escribir una definición de canalización

Definimos una función para crear una canalización de TFX. Un objeto `Pipeline` representa una canalización de TFX que se puede ejecutar con uno de los sistemas de orquestación de canalizaciones compatible con TFX.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],

      # NEW: Pass transform_graph to the trainer.
      transform_graph=transform.outputs['transform_graph'],

      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator,

      transform,  # NEW: Transform component was added to the pipeline.

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## Cómo ejecutar la canalización

Usaremos `LocalDagRunner` como en el tutorial anterior.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

Debería ver el mensaje "INFO:absl:Component Pusher is finished" (INFO:absl:Component Pusher finalizó) si la canalización se completó correctamente.

El componente Pusher inserta el modelo entrenado en `SERVING_MODEL_DIR`, que es el directorio `serving_model/penguin-transform` si no cambió las variables en los pasos anteriores. Puede ver el resultado desde el explorador de archivos en el panel del lado izquierdo de Colab, o si usa el siguiente comando:

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

También puede verificar la firma del modelo generado usando la [herramienta `saved_model_cli`](https://www.tensorflow.org/guide/saved_model#show_command).

In [ ]:
!saved_model_cli show --dir {SERVING_MODEL_DIR}/$(ls -1 {SERVING_MODEL_DIR} | sort -nr | head -1) --tag_set serve --signature_def serving_default

Debido a que definimos `serving_default` con nuestra propia función `serve_tf_examples_fn`, la firma muestra que requiere una sola cadena. Esta cadena es una cadena serializada de tf.Examples y se parsea con la función [tf.io.parse_example()](https://www.tensorflow.org/api_docs/python/tf/io/parse_example) como definimos anteriormente (obtenga más información sobre tf.Examples [aquí](https://www.tensorflow.org/tutorials/load_data/tfrecord)).

Podemos cargar el modelo exportado y probar algunas inferencias con algunos ejemplos.

In [ ]:
# Find a model with the latest timestamp.
model_dirs = (item for item in os.scandir(SERVING_MODEL_DIR) if item.is_dir())
model_path = max(model_dirs, key=lambda i: int(i.name)).path

loaded_model = tf.keras.models.load_model(model_path)
inference_fn = loaded_model.signatures['serving_default']

In [ ]:
# Prepare an example and run inference.
features = {
  'culmen_length_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
  'culmen_depth_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[16.1])),
  'flipper_length_mm': tf.train.Feature(int64_list=tf.train.Int64List(value=[213])),
  'body_mass_g': tf.train.Feature(int64_list=tf.train.Int64List(value=[5400])),
}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

Se espera que el tercer elemento, que corresponde a la especie 'Papúa', sea el más grande de los tres.

## Siguientes pasos

Si desea obtener más información sobre el componente Transform, consulte la [guía del componente Transform](https://www.tensorflow.org/tfx/guide/transform). Puede encontrar más recursos en https://www.tensorflow.org/tfx/tutorials.

Consulte [Explicación de las canalizaciones de TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para obtener más información sobre varios conceptos en TFX.
